In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s [%(process)d] %(levelname)-8s "
                    "%(name)s,%(lineno)s\t%(message)s")
logging.getLogger().setLevel('DEBUG')

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tqdm import tqdm_notebook as tqdm

In [ ]:
# Read information to connect to the database and put it in environment variables
import os
with open('ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()

In [ ]:
db_name = 'ticclat'
#db_name = 'ticclat_test'
os.environ['dbname'] = db_name

In [ ]:
from ticclat.ticclat_schema import corpusId_x_documentId, TextAttestation, Lexicon, Wordform, Anahash, Document, Corpus, WordformLink, WordformLinkSource, lexical_source_wordform

from ticclat.dbutils import get_session, session_scope

Session = get_session(os.environ['user'], os.environ['password'], os.environ['dbname'])

In [ ]:
from ticclat.queries import wfs_min_num_lexica

with session_scope(Session) as session:
    r = wfs_min_num_lexica(session)

In [ ]:
for row in r:
    print(row)

In [ ]:
from ticclat.queries import count_unique_wfs_in_corpus

with session_scope(Session) as session:
    r = count_unique_wfs_in_corpus(session, corpus_name='SoNaR-500')

In [ ]:
print(r.fetchall())

In [ ]:
from ticclat.queries import wordform_in_corpus_over_time

def wf_frequencies(session, wf, corpus_name):
    r = wordform_in_corpus_over_time(session, wf=word, corpus_name=corpus_name)

    records = [row for row in r.fetchall()]
    df = pd.DataFrame.from_records(records, columns=['wordform_id', 'wordform', 'pub_year', 'document_frequency', 'term_frequency'])
    df.sort_values(by=['pub_year'], inplace=True)
    df['term_frequency'] = df['term_frequency'].astype(int)
    
    return df


word = 'regeering'
corpus_name='Staten Generaal Digitaal'
with session_scope(Session) as session:
    df = wf_frequencies(session, word, corpus_name)
df.plot(x='pub_year', y=['term_frequency', 'document_frequency'], figsize=(10,5), grid=True, title=f'\"{word}\" in {corpus_name}')

In [ ]:
word = 'regering'
corpus_name='Staten Generaal Digitaal'
with session_scope(Session) as session:
    df2 = wf_frequencies(session, word, corpus_name)
df2.plot(x='pub_year', y=['term_frequency', 'document_frequency'], figsize=(10,5), grid=True, title=f'\"{word}\" in {corpus_name}')

In [ ]:
from ticclat.queries import wordform_in_corpora

word = 'regering'
corpus_name='Staten Generaal Digitaal'

with session_scope(Session) as session:
    r = wordform_in_corpora(session, wf=word)
records = [row for row in r]
df = pd.DataFrame.from_records(records, columns=['wordform_id', 'wordform', 'pub_year', 'document_frequency', 'term_frequency'])
df.sort_values(by=['pub_year'], inplace=True)
df['term_frequency'] = df['term_frequency'].astype(int)
df.plot(x='pub_year', y=['term_frequency', 'document_frequency'], figsize=(10,5), grid=True, title=f'\"{word}\" in {corpus_name}')

In [ ]:
records = [row for row in r]
df = pd.DataFrame.from_records(records)
df

In [ ]:
df

In [ ]:
from sqlalchemy import select, text
from sqlalchemy.sql import func, distinct, and_, desc

def wordform_in_corpora_over_time(session, wf):
    """Given a wordform, and a corpus, return word frequencies over time.

    Gives both the term frequency and document frequency.
    """
    q = select([Corpus.name, Document.pub_year,
                func.count(Document.document_id).label('document_frequency'),
                func.sum(TextAttestation.frequency).label('term_frequency'),
                func.sum(Document.word_count).label('num_words')]) \
        .select_from(Corpus.__table__.join(corpusId_x_documentId,
                                           Corpus.corpus_id ==
                                           corpusId_x_documentId.c.corpus_id)
                     .join(Document,
                           Document.document_id ==
                           corpusId_x_documentId.c.document_id)
                     .join(TextAttestation).join(Wordform)) \
        .where(Wordform.wordform == wf) \
        .group_by(Corpus.name, Document.pub_year, Wordform.wordform, Wordform.wordform_id)

    print(f'Executing query:\n{q}')

    return pd.read_sql(q, session.connection())

with session_scope(Session) as session:
    r = wordform_in_corpora_over_time(session, wf='regering')
r

In [ ]:
r['normalized_term_frequency'] = r['term_frequency'] / r['num_words'] * 100.0
r

In [ ]:
import holoviews as hv
from IPython.display import HTML
hv.notebook_extension()

In [ ]:
data = hv.Dataset(hv.Table(r[['name', 'pub_year', 'normalized_term_frequency']]), ['name', 'pub_year'], ['normalized_term_frequency'])

In [ ]:
print(data)

In [ ]:
data.to(hv.Curve, 'pub_year', 'normalized_term_frequency')

In [ ]:
ndoverlay = data.select().to(hv.Curve, 'pub_year', 'normalized_term_frequency').overlay('name')
ndoverlay